In [1]:
from collections import defaultdict
from __future__ import annotations

In [2]:
class Simplex:
    def __init__(self, vertices: list[int]):
        self.n = len(vertices) - 1
        self.vertices = tuple(sorted(vertices))

    def get_immediate_children(self) -> list[Simplex]:
        if self.n == 0:
            return []
        return [
            Simplex([*self.vertices[:i], *self.vertices[i + 1 :]])
            for i in range(self.n + 1)
        ]

    def __str__(self) -> str:
        return f"Simplex({' '.join(str(v) for v in self.vertices)})"

    def __repr__(self) -> str:
        return self.__str__()

    def __eq__(self, other: Simplex) -> bool:
        return self.vertices == other.vertices

    def __hash__(self) -> int:
        return hash(self.vertices)


a = Simplex([1, 2, 3])
print(a)
print(a.get_immediate_children())

Simplex(1 2 3)
[Simplex(2 3), Simplex(1 3), Simplex(1 2)]


In [3]:
class SimplicialComplex:
    def __init__(self, simplices: list[Simplex]):
        # It will be useful later to keep the simplices graded by dimension.
        self.graded_simplices = defaultdict(list)
        for simplex in simplices:
            self.graded_simplices[simplex.n].append(simplex)
        self.largest_n = max(self.graded_simplices.keys())
        self.fill_in_gaps()
        self.remove_duplicates()
        self.sort_simplices()

    def fill_in_simplex(self, simplex: Simplex):
        if simplex in self.graded_simplices[simplex.n]:
            return
        self.graded_simplices[simplex.n].append(simplex)
        for child in simplex.get_immediate_children():
            self.fill_in_simplex(child)

    def fill_in_gaps(self):
        for n in range(1, self.largest_n + 1):
            for simplex in self.graded_simplices[n]:
                for child in simplex.get_immediate_children():
                    self.fill_in_simplex(child)

    def remove_duplicates(self):
        for n in range(self.largest_n + 1):
            self.graded_simplices[n] = list(set(self.graded_simplices[n]))

    def sort_simplices(self):
        for n in range(self.largest_n + 1):
            self.graded_simplices[n] = sorted(
                self.graded_simplices[n], key=lambda simplex: simplex.vertices
            )

    def __str__(self) -> str:
        return f"SimplicialComplex({dict(self.graded_simplices)})"

    def __repr__(self) -> str:
        return self.__str__()

    def get_boundary_matrix_map(self) -> dict[int, list[list[float]]]:
        boundary_matrix_map = {}
        for n in range(1, self.largest_n + 1):
            curr_basis = self.graded_simplices[n]
            prev_basis = self.graded_simplices[n - 1]
            boundary_matrix = [
                [0 for _ in range(len(curr_basis))] for _ in range(len(prev_basis))
            ]
            for i in range(len(prev_basis)):
                for j in range(len(curr_basis)):

                    # Check if all vertices of the i-th simplex are in the j-th simplex.
                    if not all(
                        v in curr_basis[j].vertices for v in prev_basis[i].vertices
                    ):
                        boundary_matrix[i][j] = 0
                    else:
                        missing_index = [
                            index
                            for index, v in enumerate(curr_basis[j].vertices)
                            if v not in prev_basis[i].vertices
                        ][0]
                        boundary_matrix[i][j] = (-1) ** missing_index
            boundary_matrix_map[n] = boundary_matrix

        return boundary_matrix_map


a, b, c = Simplex([1, 2, 3]), Simplex([2, 3]), Simplex([3, 4])
X = SimplicialComplex([a, c, b])
X

SimplicialComplex({2: [Simplex(1 2 3)], 1: [Simplex(1 2), Simplex(1 3), Simplex(2 3), Simplex(3 4)], 0: [Simplex(1), Simplex(2), Simplex(3), Simplex(4)]})

In [4]:
for n in range(4):
    print(f"Simplices of dimension {n}: {X.graded_simplices[n]}")
for n, matrix in X.get_boundary_matrix_map().items():
    print(f"Boundary matrix of dimension {n}:")
    for row in matrix:
        print(row)

Simplices of dimension 0: [Simplex(1), Simplex(2), Simplex(3), Simplex(4)]
Simplices of dimension 1: [Simplex(1 2), Simplex(1 3), Simplex(2 3), Simplex(3 4)]
Simplices of dimension 2: [Simplex(1 2 3)]
Simplices of dimension 3: []
Boundary matrix of dimension 1:
[-1, -1, 0, 0]
[1, 0, -1, 0]
[0, 1, 1, -1]
[0, 0, 0, 1]
Boundary matrix of dimension 2:
[1]
[-1]
[1]
[0]
